***
# **Diplomová práce - výpočty**
***
### **Jméno:** Alois Primes
### **Název práce:** MODULARNÍ HORIZONTÁLNÍ KOTEL – HRSG
### **Zadání**
**Stručná charakteristika problematiky úkolu:**

Práce je zaměřena na návrh horizontálního kotle využívajícího spaliny za plynovou spalovací turbínou pro produkci páry k dalšímu energetickému využití. Kotel by měl být moderní konstrukce, zahrnující
maximální prefabrikaci transportních bloků (modulů), která je ovlivněna podmínkami dopravního profilu na místo stavby. Kotel bude řešen jako modulární, s horizontálním prouděním spalin, konvekčním
výparníkem, výhřevnými plochami ohříváku vody (ekonomizéry) a přehříváky páry s regulací teploty páry vstřikem napájecí vody, odebírané mezi dvěma ekonomizéry.

**Základní parametry kotle jsou:** parní výkon 76 t/h, tlak 3,1 MPa, výstupní teplota 370 °C. Teplota
napájecí vody kotle je 71°C. Další parametry kotle a odpadních spalin budou upřesněny v rámci
realizace práce.

**Cíle diplomové práce:**

Zpracovat tepelný výpočet kotle.
Zpracovat rozměrový návrh kotle, včetně základní výkresové dokumentace kotle a dispozičního
řešení.
Vyčíslit dosaženou účinnost kotle při vztažné teplotě 25 °C, použijte přímou metodu.
Zpracovat diagram průběhu teploty pracovního média a spalin a stanovit bilanci vstřiku.

In [58]:
# Instalace a aktualizace modulů

# !pip install --upgrade pip
# !pip install --upgrade tabulate
# !pip install --upgrade forallpeople
# !pip install --upgrade handcalcs
# !pip install --upgrade pytexit
# !pip install --upgrade pyXSteam
# !pip install --upgrade CoolProp

In [59]:
# Import knihoven a modulů

import pandas as pd
from CoolProp.CoolProp import PropsSI
import handcalcs.render
from math import pi, e, sqrt
from pyXSteam.XSteam import XSteam
from tabulate import tabulate
import pyromat as pm

In [60]:
# Nastavení prostředí a modulů

%decimal_separator ,
steam = XSteam(XSteam.UNIT_SYSTEM_MKS) # Jednotky: m/kg/sec/°C/bar/W
pd.set_option("display.precision", 1)

pm.config['unit_temperature'] = 'C'

***
## Základní parametry kotle
***

In [61]:
%%render params 3
m_p = (76*1000)/3600        #kg/s
p_out = 3.1                 #MPa
t_out = 370                 #°C
t_nv = 71                   #°C
m_vst = 0.36                #kg/s
t_vst = 217                 #°C

<IPython.core.display.Latex object>

***
## Parametry spalin ze spalovací turbíny
***

In [62]:
R = 8.314462618 #J/kg/mol
T = 273.15 #K

#### koeficienty polynomů pro výpočet entalpií spalin

In [63]:
# N2
a_1_N2 = 0.3694063e+01
a_2_N2 = -0.1334139e-02
a_3_N2 = 0.2652067e-05
a_4_N2 = -0.9775312e-09
a_5_N2 = -0.9983837e-13
a_6_N2 = -0.9758881e+03

# O2
a_1_O2 = 0.3721461e+01
a_2_O2 = -0.2518398e-02
a_3_O2 = 0.8589429e-05
a_4_O2 = -0.8305377e-08
a_5_O2 = 0.2710013e-11
a_6_O2 = -0.9701834e+03

# CO2
a_1_CO2 = 0.2171539e+01
a_2_CO2 = 0.1038499e-01
a_3_CO2 = -0.1074106e-04
a_4_CO2 = 0.6350127e-08
a_5_CO2 = -0.1629149e-11
a_6_CO2 = -0.9159472e+03

# Ar
a_1_Ar = 2.501658
a_2_Ar = 0
a_3_Ar = 0
a_4_Ar = 0
a_5_Ar = 0
a_6_Ar = -683.328

# H2O
a_1_H2O = 4.159259
a_2_H2O = -0.001725577
a_3_H2O = 5.70201e-06
a_4_H2O = -4.59605e-09
a_5_H2O = 1.42431e-12
a_6_H2O = -1104.501


#### Objemové složení spalin, hmotnostní průtok a teplota na vstupu do hořáku

In [64]:
%%render params 3
m_sp = 130.1    #kg/s
t_sp = 562.0    #°C

# Objemové složení spalin
x_N2 = 0.7457
x_O2 = 0.1289
x_H2O = 0.0801
x_CO2 = 0.0364
x_Ar = 0.0089

<IPython.core.display.Latex object>

#### Molární hmotnosti spalin

In [65]:
%%render params 3

# Molární hmotnost složek spalin (z podkladů)
M_N2 = 28.0134e-03      #kg/kmol
M_O2 = 31.9988e-03      #kg/kmol
M_H2O = 18.0152e-03     #kg/kmol
M_CO2 = 44.0095e-03     #kg/kmol
M_Ar = 39.9480e-03      #kg/kmol

<IPython.core.display.Latex object>

#### Hustota složek spalin (z podkladů)

In [66]:
%%render params
rho_N2 = 1.2504     #kg/m3
rho_O2 = 1.4290     #kg/m3
rho_H2O = 0.8038    #kg/m3
rho_CO2 = 1.9772    #kg/m3
rho_Ar = 1.7840     #kg/m3
rho_sp = x_N2*rho_N2 + x_O2*rho_O2 + x_H2O*rho_H2O + x_CO2*rho_CO2 + x_Ar*rho_Ar #kg/m3

<IPython.core.display.Latex object>

#### Hmotnostní procenta (-)

In [67]:
# Hmotnosti
m_N2 = x_N2*rho_N2                      #kg
m_O2 = x_O2*rho_O2                      #kg
m_H2O = x_H2O*rho_H2O                   #kg
m_CO2 = x_CO2*rho_CO2                   #kg
m_Ar = x_Ar*rho_Ar                      #kg

In [68]:
# Hmotnostní podíl
w_N2 = m_N2/rho_sp                        #kg
w_O2 = m_O2/rho_sp                        #kg
w_H2O = m_H2O/rho_sp                      #kg
w_CO2 = m_CO2/rho_sp                      #kg
w_Ar = m_Ar/rho_sp                        #kg

#### Objemový průtok spalin

In [69]:
%%render
V_sp = m_sp/rho_sp #m3n/s

<IPython.core.display.Latex object>

In [70]:
%%render params 3

# Měrný objem složek spalin (z podkladů)
v_N2 = 22.403     #m3/kg
v_O2 = 22.392     #m3/kg
v_H2O = 22.414    #m3/kg
v_CO2 = 22.258    #m3/kg
v_Ar = 22.393     #m3/kg

<IPython.core.display.Latex object>

#### Entalpie spalin (z modulu PyroMat - shodné s poskytnutými materiály)

In [71]:
N2 = pm.get('ig.N2')
O2 = pm.get('ig.O2')
H2O = pm.get('ig.H2O')
CO2 = pm.get('ig.CO2')
Ar = pm.get('ig.Ar')

In [72]:
def entalpie_sp(teplota, jednotka):

    """
    teplota = zadávat ve °C
    jednotka = 'kg' / 'm3n'

    Vypočítá entalpii [kJ/kg nebo kJ/m3n] spalin při dané teplotě [°C] vzhledem k entalpii daného prvku při 0°C. Výstupem je dictionary. Pro zobrazení entalpie dané teploty je nutné vybrat daný prvek nebo spaliny ['N2', 'O2', 'H2O', 'CO2', 'Ar', 'sp'].
    """

    h_N2 = float(N2.h(teplota)-N2.h(0)) #kJ/kg
    I_N2 = h_N2*rho_sp

    h_O2 = float(O2.h(teplota)-O2.h(0)) #kJ/kg
    I_O2 = h_O2*rho_sp

    h_H2O = float(H2O.h(teplota)-H2O.h(0)) #kJ/kg
    I_H2O = h_H2O*rho_sp

    h_CO2 = float(CO2.h(teplota)-CO2.h(0)) #kJ/kg
    I_CO2 = h_CO2*rho_sp

    h_Ar = float(Ar.h(teplota)-Ar.h(0)) #kJ/kg
    I_Ar = h_Ar*rho_sp

    h_spalin = w_N2*h_N2 + w_O2*h_O2 + w_H2O*h_H2O + w_CO2*h_CO2 + w_Ar*h_Ar #kJ/kg
    I_spalin = w_N2*I_N2 + w_O2*I_O2 + w_H2O*I_H2O + w_CO2*I_CO2 + w_Ar*I_Ar #kJ/m3n


    if jednotka == 'kg':
        return {'N2': h_N2, 'O2': h_O2, 'H2O': h_H2O , 'CO2': h_CO2, 'Ar': h_Ar, 'sp': h_spalin}
    elif jednotka == 'm3n':
        return {'N2': I_N2, 'O2': I_O2, 'H2O': I_H2O , 'CO2': I_CO2, 'Ar': I_Ar, 'sp': I_spalin}
    else:
        print('Nutno zadat jednotku - kg / m3n!')

### <u>Bod A (Vstup spalin - za hořákem)

In [73]:
%%render
t_A = t_sp #°C
I_A_sp = entalpie_sp(t_A, 'm3n')['sp']  #kJ/m3n

<IPython.core.display.Latex object>

### Bod B (za druhým přehřívákem)

In [74]:
%%render
t_B = 549 #°C
I_B_sp = entalpie_sp(t_B, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod C (za prvním přehřívákem)

In [75]:
%%render
t_C = 507 #°C
I_C_sp = entalpie_sp(t_C, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod D (za druhým výparníkem)

In [76]:
%%render
t_D = 366 #°C
I_D_sp = entalpie_sp(t_D, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod E (za prvním výparníkem)

In [77]:
%%render
t_E = 254 #°C
I_E_sp = entalpie_sp(t_E, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod F (za druhým ekonomizérem)

In [78]:
%%render
t_F = 241 #°C
I_F_sp = entalpie_sp(t_F, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod G (za prvním ekonomizérem, výstup spalin)

In [79]:
%%render
t_G = 145 #°C
I_G_sp = entalpie_sp(t_G, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

***
## Výpočty ze strany vody a vodní páry
***

#### Bod 1 (Konec druhého přehříváku, výstupní pára)

In [81]:
%%render params
t_1 = t_out                     #°C
p_1 = p_out                     #MPa
h_1 = steam.h_pt(p_1*10,t_1)    #kJ/kg
v_1 = steam.v_pt(p_1*10,t_1)    #m3/kg
x_1 = steam.x_ph(p_1*10,i_1)    #-

<IPython.core.display.Latex object>

#### Bod 3 (Konec prvního přehříváku, před vstřikem)

In [82]:
%%render params
t_3 = 348                       #°C
p_3 = 3.33                      #MPa
h_3 = steam.h_pt(p_3*10,t_3)    #kJ/kg
v_3 = steam.v_pt(p_3*10,t_3)    #m3/kg
x_3 = steam.x_ph(p_3*10,i_3)    #-

<IPython.core.display.Latex object>

#### Bod 4 (Začátek prvního přehříváku, po druhém výparníku)

In [83]:
%%render params
p_4 = 3.38                      #MPa
t_4 = steam.tsat_p(p_4*10)      #°C
h_4 = steam.hV_p(p_4*10)        #kJ/kg
v_4 = steam.vV_p(p_4*10)        #m3/kg
x_4 = steam.x_ph(p_4*10,i_4)    #-

<IPython.core.display.Latex object>

#### Bod 5 (začátek prvního výparníku, kde je sytá kapalina)

In [84]:
%%render params
p_5 = 3.38                      #MPa
t_5 = steam.tsat_p(p_5*10)      #°C
h_5 = steam.hL_p(p_5*10)        #kJ/kg
v_5 = steam.vL_p(p_5*10)        #m3/kg
x_5 = steam.x_ph(p_5*10,i_5)    #-

<IPython.core.display.Latex object>

#### Bod 6 (Za posledním ekonomizérem, před vstupem do výparníku)

In [85]:
%%render params
p_6 = 3.68                      #MPa
t_6 = 236                       #°C
h_6 = steam.h_pt(p_6*10, t_6)   #kJ/kg
v_6 = steam.v_pt(p_6*10, t_6)   #m3/kg
x_6 = steam.x_ph(p_6*10,i_6)    #-

<IPython.core.display.Latex object>

#### Bod 7 (mezi ekonomizéry)

In [86]:
%%render params
p_7 = 3.74                      #MPa
t_7 = 217                       #°C
h_7 = steam.h_pt(p_7*10, t_7)   #kJ/kg
v_7 = steam.v_pt(p_7*10, t_7)   #m3/kg
x_7 = steam.x_ph(p_7*10,i_7)    #-


<IPython.core.display.Latex object>

#### Bod 2 (Začátek druhého přehříváku, po vstřiku)

In [87]:
%%render params
p_2 = p_3                           #MPa
h_2 = (20.74*i_3 + 0.36*i_7)/21.10  #kJ/kg
t_2 = steam.t_ph(p_2*10, i_2)       #°C
v_2 = steam.v_ph(p_2*10, i_2)       #m3/kg
x_2 = steam.x_ph(p_2*10,i_2)        #-

<IPython.core.display.Latex object>

#### Bod 8 (Před začátkem ekonomizéru, vstup napájecí vody)

In [111]:
%%render params
t_8 = 71                         #°C
p_8 = 4.2                        #MPa
h_8 = steam.h_pt(p_8*10,t_8)     #kJ/kg
v_8 = steam.v_pt(p_8*10,t_8)     #m3/kg
x_8 = steam.x_ph(p_8*10,i_8)     #-

<IPython.core.display.Latex object>

### Tabulka bodová pro vodu a páru

In [89]:
bodova_tabulka = pd.DataFrame(
    {
        "T [°C]": (t_1, t_2, t_3, t_4, t_5, t_6, t_7, t_8),
        "p [MPa]": (p_1, p_2, p_3, p_4, p_5, p_6, p_7, p_8),
        "h [kJ/kg]": (h_1, h_2, h_3, h_4, h_5, h_6, h_7, h_8),
        "v [m3/kg]": (v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8),
        "x [-]": (x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8),
    },
    index = ['1','2','3','4','5','6','7','8']
)

In [90]:
bodova_tabulka

# print(bodova_tabulka.to_latex())

,T [°C],p [MPa],h [kJ/kg],v [m3/kg],x [-]
1,370.0,3.1,3160.7,9.1e-02,1.0
2,332.7,3.3,3066.8,7.8e-02,1.0
3,348.0,3.3,3103.9,8.1e-02,1.0
4,240.6,3.4,2803.0,5.9e-02,1.0
5,240.6,3.4,1040.2,1.2e-03,0.0
6,236.0,3.7,1018.6,1.2e-03,0.0
7,217.0,3.7,930.3,1.2e-03,0.0
8,71.0,4.0,300.4,1.0e-03,0.0


***
## Výkon kotle a jednotlivých teplosměnných ploch
***

### Maximální využitelný výkon

In [91]:
%%render
# Dodaný výkon v palivu
Q_in = V_sp*(I_A_sp-entalpie_sp(25, 'm3n')['sp']) #kW

<IPython.core.display.Latex object>

In [114]:
%%render
# Odebraný výkon vodou a párou
Q_out = m_p*h_1-m_p*h_8 #kW

<IPython.core.display.Latex object>

### Účinnost

In [116]:
%%render
eta = Q_out/Q_in #-

<IPython.core.display.Latex object>

### Tepelné ztráty (konvekce a sálání)

In [93]:
%%render
C = 0.0113
Q_RC = Q_N**0.7*C #MW

<IPython.core.display.Latex object>

#### Poměrná ztráta sáláním a konvekcí

In [94]:
%%render
z_s = Q_RC/Q_N #-

<IPython.core.display.Latex object>

### Tepelny vykon odevzdany spalinami mezi body A-E

In [95]:
%%render
Q_AE = (I_A_sp - I_E_sp)*V_sp #kW

<IPython.core.display.Latex object>

### Tepelny vykon prijaty vodou mezi body 1-6

In [96]:
%%render
Q_16 = Q_AE*(1-z_s) #kW

<IPython.core.display.Latex object>

### Predane tepelne toky v jednotlivych teplosmennych plochach

#### Prehrivak 2

In [97]:
%%render
Q_P2 = m_p*(i_1-i_2) #kW

<IPython.core.display.Latex object>

#### Prehrivak 1

In [98]:
%%render
Q_P1 = (m_p-m_vst)*(i_3-i_4) #kW

<IPython.core.display.Latex object>

#### Vyparnik 1 + 2

In [99]:
%%render
Q_V = (m_p-m_vst)*(i_4-i_5) #kW

<IPython.core.display.Latex object>

#### Ekonomizer 2

In [100]:
%%render
Q_EKO_2 = (m_p-m_vst)*(i_6-i_7) #kW

<IPython.core.display.Latex object>

#### Ekonomizer 1

In [101]:
%%render
Q_EKO_1 = m_p*(i_7-i_8) #kW

<IPython.core.display.Latex object>

***
SPATNE, NEVYCHAZI KONTROLA - JSOU SPATNE ENTALPIE (tabulka entalpii spalin ze semestralky ma odlisne hodnoty ==> po dosazeni hodnot ze semestralky kontrola vychazi, STALE ALE NUTNE ZJISTIT, KDE JE PROBLEM)
***

#### Kontrolni vypocet

In [102]:
%%render
Q_16
Q_kontrolni = Q_V + Q_P1 + Q_P2

<IPython.core.display.Latex object>

***
# Návrh teplosměnných ploch
***

**Označení veličin**

D = vnější průměr

t<sub>tr</sub> = tloušťka trubky

d = vnitřní průměr

h<sub>ž</sub> = výška žebra

t<sub>ž</sub> = tloušťka žebra

n<sub>ž</sub> = počet žeber na 1 m<sup>2</sup>

s<sub>1</sub> = příčná rozteč

s<sub>2</sub> = podélná rozteč

## Přehřívák 2 (bez žebrování)
***

In [103]:
%%render params
D = 0.0445      #m
t_tr = 0.0029   #m
d = D-2*t_tr    #m
s_1 = 0.088     #m
s_2 = 0.117     #m

<IPython.core.display.Latex object>

### Rozměry trubek

Zvolená rychlost

In [104]:
%%render params
w_p = 22    #m/s

<IPython.core.display.Latex object>

Střední hodnoty

In [105]:
%%render
t_12 = (t_1+t_2)/2  #°C
p_12 = (p_1+p_2)/2  #MPa

<IPython.core.display.Latex object>

In [106]:
%%render params
v_12 = steam.v_pt(p_12*10,t_12)    #m3/kg

<IPython.core.display.Latex object>

Průtočný průřez

In [107]:
%%render
S_p = (m_p*v_12)/w_p    #m2

<IPython.core.display.Latex object>

Počet trubek

In [108]:
%%render
n_tr = (4*S_p)/(pi*d**2)
n_tr = int(n_tr)

<IPython.core.display.Latex object>

Skutečná rychlost páry

In [109]:
%%render
w_p_skut = (m_p*v_12*4)/(n_tr*pi*d**2)  #m/s

<IPython.core.display.Latex object>